#### MP based asynchronous testing

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import multiprocessing as mp
import TestPool_Unit
from shutil import copyfile
import numpy as np

/home/rs/15CS91R05/Documents/anaconda3/envs/py3_bishal/lib/python3.6/site-packages/matplotlib/font_manager.py:280: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  'Matplotlib is building the font cache using fc-list. '


In [3]:
from collections import defaultdict

In [4]:
def Evaluate(result_arr):
    print('Files Processed: ', len(result_arr))
    recalls = []
    recalls_of_word = []
    precisions = []
    precisions_of_words = []
    fully_Correct_l = 0
    fully_Correct_w = 0
    for entry in result_arr:
        (word_match, lemma_match, n_dcsWords, n_output_nodes) = entry
        recalls.append(lemma_match/n_dcsWords)
        recalls_of_word.append(word_match/n_dcsWords)

        precisions.append(lemma_match/n_output_nodes)
        precisions_of_words.append(word_match/n_output_nodes)
        if lemma_match == n_dcsWords:
            fully_Correct_l += 1
        if word_match == n_dcsWords:
            fully_Correct_w += 1
    print('Avg. Micro Recall of Lemmas: {}'.format(np.mean(np.array(recalls))))
    print('Avg. Micro Recall of Words: {}'.format(np.mean(np.array(recalls_of_word))))
    print('Avg. Micro Precision of Lemmas: {}'.format(np.mean(np.array(precisions))))
    print('Avg. Micro Precision of Words: {}'.format(np.mean(np.array(precisions_of_words))))
    rl = np.mean(np.array(recalls))
    pl = np.mean(np.array(precisions))
    print('F-Score of Lemmas: ', (2*pl*rl)/(pl+rl))
    print('Fully Correct Lemmawise: {}'.format(fully_Correct_l/len(recalls_of_word)))
    print('Fully Correct Wordwise: {}'.format(fully_Correct_w/len(recalls_of_word)))
    print('[{:0.2f}, {:0.2f}, {:0.2f}, {:0.2f}, {:0.2f}, {:0.2f}, {:0.2f}]'.format(100*np.mean(np.array(recalls)), 100*np.mean(np.array(recalls_of_word)), 100*np.mean(np.array(precisions)), \
           100*np.mean(np.array(precisions_of_words)), 100*(2*pl*rl)/(pl+rl), 100*fully_Correct_l/len(recalls_of_word),\
           100*fully_Correct_w/len(recalls_of_word)))

In [5]:
modelFile = 'outputs/train_t2753954441900/nnet_e1_i400.p' #PM2
_dump = False
if _dump:
    _outFile = 'outputs/dump_predictions/PM2_NLoss'
else:
    _outFile = None

# Backup the model file
copyfile(modelFile, modelFile + '.bk')

# Create Queue, Result array
queue = mp.Queue()
result_arr = []

# Start 6 workers - 8 slows down the pc
proc_count = 4
procs = [None]*proc_count
for i in range(proc_count):
    vpid = i
    procs[i] = mp.Process(target = TestPool_Unit.pooled_Test, args = \
                          (modelFile, vpid, queue, '../NewData/skt_dcs_DS.bz2_4K_pmi_mir_heldout/', int(9600/proc_count), _dump, _outFile))
# Start Processes
for i in range(proc_count):
    procs[i].start()

Child process with vpid:0, pid:10934 started.
Child process with vpid:1, pid:10935 started.
Child process with vpid:3, pid:10941 started.
Child process with vpid:2, pid:10938 started.
Keep Prob = 0.6, Dropout = 0.4
Keep Prob = 0.6, Dropout = 0.4
Keep Prob = 0.6, Dropout = 0.4
Keep Prob = 0.6, Dropout = 0.4
vpid:3: Range is 7200 -> 9600 / 9577
vpid:2: Range is 4800 -> 7200 / 9577
vpid:0: Range is 0 -> 2400 / 9577
vpid:1: Range is 2400 -> 4800 / 9577
Child process with vpid:1, pid:10935 closed.


Process Process-4:
Traceback (most recent call last):
  File "/home/rs/15CS91R05/Documents/anaconda3/envs/py3_bishal/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/home/rs/15CS91R05/Documents/anaconda3/envs/py3_bishal/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/rs/15CS91R05/Bishal/Clean_n_clear/TestPool_Unit.py", line 34, in pooled_Test
    fn = TestFiles[i]
IndexError: list index out of range


Child process with vpid:2, pid:10938 closed.
Child process with vpid:0, pid:10934 closed.


In [20]:
# Fetch partial results
while not queue.empty():
    result_arr.append(queue.get())
# Evaluate results till now
Evaluate(result_arr)

Files Processed:  9577
Avg. Micro Recall of Lemmas: 0.8488105596019528
Avg. Micro Recall of Words: 0.7417558132070324
Avg. Micro Precision of Lemmas: 0.7834907014618754
Avg. Micro Precision of Words: 0.6853795564966576
F-Score of Lemmas:  0.814843676978
Fully Correct Lemmawise: 0.37757126448783546
Fully Correct Wordwise: 0.17615119557272632
[84.88, 74.18, 78.35, 68.54, 81.48, 37.76, 17.62]


In [7]:
# Check status
for i in range(proc_count):
    p = procs[i]
    print('Process with\t vpid: {}\t ->\t pid: {}\t ->\t running status: {}'.format(i, p.pid, p.is_alive()))

Process with	 vpid: 0	 ->	 pid: 10934	 ->	 running status: True
Process with	 vpid: 1	 ->	 pid: 10935	 ->	 running status: True
Process with	 vpid: 2	 ->	 pid: 10938	 ->	 running status: True
Process with	 vpid: 3	 ->	 pid: 10941	 ->	 running status: True


In [21]:
# Properly Join
for i in range(proc_count):
    procs[i].join()

In [26]:
# Force Terminate
for p in procs:
    p.terminate()

In [24]:
np.mean([a[2] for a in result_arr])

6.7729026036644164

In [14]:
result_arr2 = [list(x) for x in result_arr]

recalls = defaultdict(list)
for i in range(len(result_arr2)):
    result_arr2[i][0] /= result_arr2[i][2]
    result_arr2[i][1] /= result_arr2[i][2]
    recalls[result_arr2[i][2]].append(result_arr2[i][1])

for c in sorted(recalls.keys()):
    print('C: {}, Samples: {}, Mean: {}'.format(c, len(recalls[c]), np.mean(recalls[c])))

C: 1, Samples: 32, Mean: 0.75
C: 2, Samples: 72, Mean: 0.8263888888888888
C: 3, Samples: 43, Mean: 0.8837209302325579
C: 4, Samples: 36, Mean: 0.8958333333333334
C: 5, Samples: 98, Mean: 0.8877551020408163
C: 6, Samples: 345, Mean: 0.9130434782608695
C: 7, Samples: 449, Mean: 0.9277760101813554
C: 8, Samples: 332, Mean: 0.916039156626506
C: 9, Samples: 112, Mean: 0.9077380952380955
C: 10, Samples: 36, Mean: 0.9249999999999999
C: 11, Samples: 19, Mean: 0.9330143540669856
C: 12, Samples: 16, Mean: 0.8854166666666667
C: 13, Samples: 6, Mean: 0.8333333333333334
C: 14, Samples: 2, Mean: 0.9642857142857143
C: 15, Samples: 4, Mean: 0.9
C: 16, Samples: 2, Mean: 0.9375
C: 17, Samples: 4, Mean: 0.9264705882352942
C: 18, Samples: 1, Mean: 0.9444444444444444
C: 22, Samples: 1, Mean: 0.6363636363636364
C: 24, Samples: 1, Mean: 0.75
C: 32, Samples: 1, Mean: 0.8125
C: 35, Samples: 1, Mean: 0.8571428571428571


In [8]:
ls outputs/dump_predictions/

BM3_NLoss_proc0.csv  lemmawise_labelled.csv    pred_1L_bigram_proc3.csv
BM3_NLoss_proc1.csv  pred_1L_bigram_proc0.csv  pred_1L_bigram_proc4.csv
BM3_NLoss_proc2.csv  pred_1L_bigram_proc1.csv  pred_1L_bigram_proc5.csv
BM3_NLoss_proc3.csv  pred_1L_bigram_proc2.csv
